In [1]:
import nltk
#nltk.download('all-corpora')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

#from gensim.models import Word2Vec
import gensim
from nltk.stem import PorterStemmer
nltk.download('stopwords')
punctuations = '''!()-[]{};:'"\<>/?@#$%^&*_~'''
stop_words = set(stopwords.words('english')) 
import pandas as pd
from nltk.parse.corenlp import CoreNLPDependencyParser
from nltk.corpus import wordnet 



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/amrutha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from DocSim import DocSim
from nltk import RegexpParser as re
from nltk import pos_tag
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
w2vpath='/home/amrutha/Documents/GoogleNews-vectors-negative300-SLIM.bin/GoogleNews-vectors-negative300-SLIM.bin'



In [3]:
w2vpath='/home/amrutha/Documents/GoogleNews-vectors-negative300-SLIM.bin/GoogleNews-vectors-negative300-SLIM.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(w2vpath,binary=True)

/home/amrutha/anaconda3/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [172]:
punkt_param = PunktParameters()
abbreviation = ['e.g']
punkt_param.abbrev_types = set(abbreviation)
tokenizer = PunktSentenceTokenizer(punkt_param)
porter = PorterStemmer()


In [173]:
ds= DocSim(model,stopwords=stop_words)
extract_flag=0

In [174]:
data= pd.read_csv('qas_dataset.csv')

In [175]:
data.head()

,Paragraph,Questions,Answer
0,Construction is the process of constructing a ...,What is the process of constructing a building...,Construction
1,Construction is the process of constructing a ...,what is manufacturing?,manufacturing typically involves mass producti...
2,Many questions regarding prime numbers remain ...,What is the application of prime numbers used ...,public-key cryptography
3,"Starting in the late 1950s, American computer ...",What is Donald Davies credited with,Davies is credited with coining the modern nam...
4,"In the 1960s, a series of discoveries, the mos...",What is another word for the Earth's upper man...,asthenosphere


In [176]:
#read the data and the question

para = open("para.txt","r").read()
para= para.lower()

question = open("question.txt","r").read()
question= question.lower()
df_para= pd.DataFrame()

def question_type(question):
    print(len(preprocess(question).split()))
    if len(preprocess(question).split())>4:
        return "non descriptive"
    else:
        return "descriptive"
    
#create the dataframe of all the sentences in the para
def create_database(para):
    global df_para
    df_para = pd.DataFrame({'sentence':[sentence for sentence in tokenizer.tokenize(para) if len(sentence)>4]})
    df_para.reset_index(level=0, inplace=True)
    
#preprocess the question and the data: remove punctuations, stop words
def preprocess(data):
    temp=[]
    for w in word_tokenize(data):
        if w not in punctuations:
            if w not in stop_words:
                
                temp.append(w)
    temp= " ".join(temp)        
    return temp
            
#calculate the cosine similarity
def similarity(sentence,question):
    vectorizer = CountVectorizer()
    sentence_vector = vectorizer.fit_transform(sentence)
    question_vector= vectorizer.transform(question)
    
    return(cosine_similarity(sentence_vector,question_vector))
    

In [177]:
#main function: to create the dataframe and to compute the similarity of the sentences.
def calculate_similarity(para,question):
      
    global df_para
    create_database(para)

    for i in range(len(df_para)):
        df_para.loc[i,'preprocessed_sent']= [preprocess(df_para['sentence'][i])]
    preprocess_question=[]
    preprocess_question.append(preprocess(question))      
    
    
    for i in range(len(df_para)):        
        df_para.loc[i,'similarity']=ds.calculate_similarity(df_para['sentence'][i],question)
        

In [178]:

    
calculate_similarity(para,question)
similar_sentence = df_para.loc[df_para['similarity'].idxmax(),'sentence']

In [179]:
similar_sentence

'primes are used in several routines in information technology, such as public-key cryptography, which makes use of properties such as the difficulty of factoring large numbers into their prime factors.'

In [180]:
sent = similar_sentence.split()
pos_similar_sentence= pos_tag(sent)

In [181]:
pos_similar_sentence

[('primes', 'NNS'),
 ('are', 'VBP'),
 ('used', 'VBN'),
 ('in', 'IN'),
 ('several', 'JJ'),
 ('routines', 'NNS'),
 ('in', 'IN'),
 ('information', 'NN'),
 ('technology,', 'NN'),
 ('such', 'JJ'),
 ('as', 'IN'),
 ('public-key', 'JJ'),
 ('cryptography,', 'NN'),
 ('which', 'WDT'),
 ('makes', 'VBZ'),
 ('use', 'NN'),
 ('of', 'IN'),
 ('properties', 'NNS'),
 ('such', 'JJ'),
 ('as', 'IN'),
 ('the', 'DT'),
 ('difficulty', 'NN'),
 ('of', 'IN'),
 ('factoring', 'VBG'),
 ('large', 'JJ'),
 ('numbers', 'NNS'),
 ('into', 'IN'),
 ('their', 'PRP$'),
 ('prime', 'JJ'),
 ('factors.', 'NN')]

In [182]:
def non_descriptive_chunker(similar_sentence):
    
    
    sent = similar_sentence.split()
    pos_similar_sentence= pos_tag(sent)
    #print(pos_similar_sentence)
   # nd_ending = "E: {<DT>?<VB.?>+<JJ.?>*<NN?>+}"
   # nd_beginning ="B: {<DT>?<JJ.?>*<NN.?>+<VBZ>+}"
    nd = """E: {<DT>?<VB.?>+<JJ.?>*<NN?>+}$
    B: ^{<DT>?<JJ.?>*<NN.?>+<VBZ>+}"""

   # re_nd_ending = re(nd_ending)
   # re_nd_beginning= re(nd_beginning)
    re_nd = re(nd)

    result = re_nd.parse(pos_similar_sentence)

    #extract the noun if its at the ending of the sentence eg: .....is called construction

    for subtree in result.subtrees(filter=lambda t: t.label()== 'E'):
        for num in range(len(subtree)):
            if subtree[num][1] in'NN':
                
                
                return(subtree[num][0])
                

        #print(' '.join(x[0] for x in subtree))

        #extract the noun if it is at the beginning of the sentence eg: Construction is.....
    for subtree in result.subtrees(filter=lambda t: t.label()=='B'):
        for num in range(len(subtree)):
            if subtree[num][1] in 'NN' and  subtree[num+1][0]=='is':
                
                print("beginning")              
                return(subtree[num][0])
            
    return None
                
   
            
    


In [183]:
non_descriptive_chunker(similar_sentence)

In [184]:

def descriptive_chunker(similar_sentence):
    sent = similar_sentence.split()
    pos_similar_sentence= pos_tag(sent)
    d_chunker ="""P1: {<DT.?>*<JJ.?>*<NN.?>+<RB.?>*<VB.?>+<NN.?>*}
    P2: {<VB.?>+<DT.?>*<JJ.?>*<NN.?>+}
    """
    re_d_chunker= re(d_chunker)
    result = re_d_chunker.parse(pos_similar_sentence)

    #result.draw()

    for subtree in result.subtrees(filter= lambda t: t.label()=='P1'):
        return(similar_sentence)
        
        #break
        #for num in range(len(subtree)):

            #x= subtree[num][0]        
            #if x in pos_q:

                #print(similar_sentence)


    for subtree in result.subtrees(filter= lambda t: t.label()=='P2'):
        return(similar_sentence)
        #for num in range(len(subtree)):
         #   if subtree[num][1] in pos_q:
          #      print(similar_sentence)



In [185]:
descriptive_chunker(similar_sentence)

'primes are used in several routines in information technology, such as public-key cryptography, which makes use of properties such as the difficulty of factoring large numbers into their prime factors.'

In [186]:
dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')


In [187]:
def convert_dep_to_df(dep):
    j=0
    dep_list=[]
    dep_list_2=[]
    for i in dep.to_conll(4).split():

        while(j<4):
            dep_list_2.append(i)
            j=j+1


            if(j==4):
                j=0
                dep_list.append(dep_list_2)
                dep_list_2=[]
                break
            else:
                break
    dependency_df= pd.DataFrame(dep_list,columns=['word','pos-tag','branch_to_index','dependency'])
    dependency_df.index+=1
    return dependency_df
       
    

In [323]:

def check_word_in_sentence(word,similar_sentence):
    #print("check word in sentence word= ",word)
    
    if word in similar_sentence:
        return word
    else:
        synonyms=[]
        for syn in wordnet.synsets(word): 
            for l in syn.lemmas(): 
                if l.name() in similar_sentence:
                    #print("synonym:", l.name())
                    return l.name()
    return None
        
        

In [355]:
def key_word_question(similar_sentence,question):
    parses= dep_parser.parse(similar_sentence.split())
    dep = parses.__next__()
    dependency_df= convert_dep_to_df(dep)
    
    parses= dep_parser.parse(question.split())
    dep_question = parses.__next__()
    dependency_q=convert_dep_to_df(dep_question)
    verbs=['VB','VBD','VBG','VBP','VBZ','VBN']
    
    
    #print(dependency_q)
    #print()
    #print(dependency_df)
    
    
    #find the root of the dependency treee
    root_index=int(dependency_df[dependency_df['dependency']=='ROOT'].index.values)
    
    #print("root_index=",root_index)
    
    
    #chech if the root word of tree is in question, if yes then return
    if(dependency_df['word'][root_index] in question and dependency_df['pos-tag'][root_index]=='NN'):
        
        #print("root index found by: root of tree==keynoun")
        return root_index
    
    
    
    # if no, check if the root word is verb, if yes, check if word exists in question
    elif(dependency_df['word'][root_index] in question and dependency_df['pos-tag'][root_index] in verbs):
        #print("verbs")
        index_in_question=int(dependency_q[dependency_q['word']==dependency_df['word'][root_index]].index.values)
    
    #if word is in question, check if there are any NMOD connections to it. If yes, return the index noun as main index
        for i in (dependency_q[dependency_q['branch_to_index']==str(index_in_question)].index.values):
            if(dependency_q['dependency'][i]=="nmod"):
                # check if the NMOD word in question is present in sentence
                
                if(check_word_in_sentence(dependency_q['word'][i],similar_sentence) is not None):
                    matched_word=check_word_in_sentence(dependency_q['word'][i],similar_sentence)
                    #print("dependency_df[dependency_df['word']==dependency_q['word'][i]].index.values")
                    #print(dependency_df[dependency_df['word']==str(matched_word)].index.values)
                    
                    root_index=int(dependency_df[dependency_df['word']==dependency_q['word'][i]].index.values)
                    print("root index found by: NMOD of verb == keynoun")
                    return root_index
        
    # if the root word is a noun but not in question, check if it has NSUBJ
    else:   
        for i in (dependency_q[dependency_q['branch_to_index']=="1"].index.values):
            
            if(dependency_q['dependency'][i]=="nsubj"):
                
                #if NSUBJ of question is in sentence, return
                if(check_word_in_sentence(dependency_q['word'][i],similar_sentence)is not None):
                    
                    matched_word=check_word_in_sentence(dependency_q['word'][i],similar_sentence)                    
                    #print("dependency_df[dependency_df['word']==dependency_q['word'][i]].index.values")
                    #print(dependency_df[dependency_df['word']==dependency_q['word'][i]].index.values)
                    root_index=int(dependency_df[dependency_df['word']==str(matched_word)].index.values)
                    print("root index found by: NSUBJ of noun== key noun")
                    return root_index
                #  if NSUBJ is not in question, check if it has ACL branch. 
                else:
                    #print("dependency_q[dependency_q['branch_to_index']==str(i)].index.values")
                    #print(dependency_q[dependency_q['branch_to_index']==str(i)].index.values)
                    for j in (dependency_q[dependency_q['branch_to_index']==str(i)].index.values):
                         if(dependency_q['dependency'][i]=="acl"):
                                if(check_word_in_sentence(dependency_q['word'][i],similar_sentence)is not None):
                                    matched_word=check_word_in_sentence(dependency_q['word'][i],similar_sentence)
                                    root_index=int(dependency_df[dependency_df['word']==str(matched_word)].index.values)
                                    print("root index found by: NSUBJ-->ACL == key noun")
                                    return root_index

    
    return root_index

In [356]:
root_index=key_word_question(similar_sentence,question)

root index found by: NMOD of verb == keynoun


In [357]:
answer=[]

In [358]:

def get_answer_dependency(similar_sentence,root_index):
    parses= dep_parser.parse(similar_sentence.split())
    dep = parses.__next__()
    dependency_df= convert_dep_to_df(dep)
    
    print("root_word:",dependency_df['word'][root_index])
    flag=0
    
   
    for i in (dependency_df[dependency_df['branch_to_index']==str(root_index)].index.values):
         #RULE 1: ACL/ADVCL-->DOBJ-->NMOD-->CONJ
        if(dependency_df['dependency'][i]=='advcl' or dependency_df['dependency'][i]=='acl' ):
            if i>root_index:
                flag=1
                new_index= i
                
            else: break
                #print("gg",new_index)
            for j in (dependency_df[dependency_df['branch_to_index']==str(new_index)].index.values):
                if(dependency_df['dependency'][j]=='dobj'):
                    if j>new_index:
                        pass
                        #print("gg",j)

                    for k in (dependency_df[dependency_df['branch_to_index']==str(j)].index.values):
                        if(dependency_df['dependency'][k]=='nmod'):
                            if k>j:
                                
                                #print(k)
                                break
            print("RULE 1: ACL/ADVCL-->DOBJ-->NMOD-->CONJ")
            return(" ".join(dependency_df['word'][new_index-1:k]))
            
        #RULE 2: (NO ACL/ADVCL/DOBJ) NMOD-->CONJ
        elif(dependency_df['dependency'][i]=='nmod' and flag==0):
            
            

            #return(" ".join(dependency_df['word'][root_index:i+1]))
            for j in (dependency_df[dependency_df['branch_to_index']==str(i)].index.values):
                
                if(dependency_df['dependency'][j]=='conj'):
                    if j>i:
                        
                        new_index=j
                        print("NO ACL/ADVCL/DOBJ) NMOD-->CONJ ")
                        return(" ".join(dependency_df['word'][j:new_index]))
                    
            if(i>root_index):
                print("RULE 2: NO ACL/ADVCL/DOBJ) NMOD")
                if(dependency_df['dependency'][i-1]=='DET' or dependency_df['dependency'][i-1]=='compound'):
                    return(" ".join(dependency_df['word'][i-2:i]))
                else:
                    return(dependency_df['word'][i])
                
    return None
               
                        
        



           



#else:
#    return(get_answer_dependency(root_index))
#    extract_flag=1
        
        
    

In [359]:
get_answer_dependency(similar_sentence,root_index)

root_word: technology
RULE 2: NO ACL/ADVCL/DOBJ) NMOD


'public-key cryptography'

In [344]:
data

,Paragraph,Questions,Answer
0,Construction is the process of constructing a ...,What is the process of constructing a building...,Construction
1,Construction is the process of constructing a ...,what is manufacturing?,manufacturing typically involves mass producti...
2,Many questions regarding prime numbers remain ...,What is the application of prime numbers used ...,public-key cryptography
3,"Starting in the late 1950s, American computer ...",What is Donald Davies credited with,Davies is credited with coining the modern nam...
4,"In the 1960s, a series of discoveries, the mos...",What is another word for the Earth's upper man...,asthenosphere
5,The development of plate tectonics provided a ...,What is the area called where two plates move ...,divergent boundaries
6,The development of plate tectonics provided a ...,What is the area called where one plate subduc...,convergent boundaries
7,Seismologists can use the arrival times of sei...,What is the outer core called?,<No Answer>
8,"During the mid-Eocene, it is believed that the...",What is the lake known as which was created by...,Solimões Basin
9,NASA's CALIPSO satellite has measured the amou...,What is the name of the satellite that measure...,CALIPSO


In [347]:

    
def models(data):
    
    for i in range(0,42):
        try:
            para=data['Paragraph'][i]
            question=data['Questions'][i]
            q_type= question_type(question)

            calculate_similarity(para,question)
            similar_sentence = df_para.loc[df_para['similarity'].idxmax(),'sentence']

            if q_type=="non descriptive":
                predicted_ans=non_descriptive_chunker(similar_sentence)
                type="non descriptive"

            elif(q_type=='descriptive'):
                root_index=key_word_question(similar_sentence,question)
                predicted_ans=get_answer_dependency(similar_sentence,root_index)
                type="dependency"


                if( predicted_ans is None):

                    predicted_ans=non_descriptive_chunker(similar_sentence)
                    type="descriptive"

            if( predicted_ans is None):
                    root_index=key_word_question(similar_sentence,question)                
                    predicted_ans=get_answer_dependency(similar_sentence,root_index)
                    type="dependency"

            print("-------------------------------------------------------------------------")
            print(type,":")
            print("question:",question)
            print("SIMILAR_SENTENCE:",similar_sentence)
            print("Ground answer:", data['Answer'][i])
            print("Answer:" ,predicted_ans)
            print("done")
            print("-------------------------------------------------------------------------")
            
        except:
            print("##############################################")
            print("EXCEPTION OCCURED")
            print("i=",i)
            print("###############################################")

        
    
                
    

In [348]:
models(data)

5
beginning
-------------------------------------------------------------------------
non descriptive :
question: What is the process of constructing a building or infrastructure?
SIMILAR_SENTENCE: Construction is the process of constructing a building or infrastructure.
Ground answer: Construction
Answer: Construction
done
-------------------------------------------------------------------------
1
root_word: process
RULE 1: ACL/ADVCL-->DOBJ-->NMOD-->CONJ
-------------------------------------------------------------------------
dependency :
question: what is manufacturing?
SIMILAR_SENTENCE: Construction is the process of constructing a building or infrastructure.
Ground answer: manufacturing typically involves mass production of similar items without a designated purchaser
Answer: constructing a building or infrastructure
done
-------------------------------------------------------------------------
14
root index found by: NMOD of verb == keynoun
root_word: technology
RULE 2: NO ACL/AD

root_word: remain
RULE 2: NO ACL/ADVCL/DOBJ) NMOD
-------------------------------------------------------------------------
dependency :
question: What is the name of the supposition that any number larger than 2 can be represented as the sum of two primes?
SIMILAR_SENTENCE: Many questions regarding prime numbers remain open, such as Goldbach's conjecture (that every even integer greater than 2 can be expressed as the sum of two primes), and the twin prime conjecture (that there are infinitely many pairs of primes whose difference is 2).
Ground answer: Goldbach's conjecture
Answer: conjecture
done
-------------------------------------------------------------------------
9
root_word: remain
RULE 2: NO ACL/ADVCL/DOBJ) NMOD
-------------------------------------------------------------------------
dependency :
question: What is the name of the supposition that there are infinite pairs of primes whose difference is 2?
SIMILAR_SENTENCE: Many questions regarding prime numbers remain open, suc

root_word: expensive
RULE 2: NO ACL/ADVCL/DOBJ) NMOD
-------------------------------------------------------------------------
dependency :
question: What is the comparison in price between Australian private schools versus public?
SIMILAR_SENTENCE: Private schools in Australia are always more expensive than their public counterparts.
Ground answer: more expensive
Answer: counterparts
done
-------------------------------------------------------------------------
4
##############################################
EXCEPTION OCCURED
i= 35
###############################################
4
root_word: includes
-------------------------------------------------------------------------
descriptive :
question: What is included with each packet label?
SIMILAR_SENTENCE: In connectionless mode each packet includes complete addressing information.
Ground answer: destination address, source address, and port numbers
Answer: information.
done
-------------------------------------------------------------